In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',500)
####teste

In [2]:
df = pd.read_csv(r'../data/train.csv')
print(df.shape)
df.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
df.Survived.value_counts(normalize = True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

In [5]:
df=df.fillna({'Age':30.00})
df.dropna(subset=['Embarked'], inplace = True)

In [6]:
from sklearn.preprocessing import OneHotEncoder
encod = OneHotEncoder()
df_encod =df[['Pclass','Sex','Embarked']].values
matrix_sparse=encod.fit_transform(df_encod).toarray() 
matrix = pd.DataFrame(matrix_sparse,columns=encod.get_feature_names())
matrix.head()


,x0_1,x0_2,x0_3,x1_female,x1_male,x2_C,x2_Q,x2_S
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [7]:
df = df.merge(matrix, how ='left',on = df.index)
df.drop(columns=['Cabin','PassengerId','Ticket','key_0','Pclass','Sex','Embarked'], inplace = True)
df.head()

,Survived,Name,Age,SibSp,Parch,Fare,x0_1,x0_2,x0_3,x1_female,x1_male,x2_C,x2_Q,x2_S
0,0,"Braund, Mr. Owen Harris",22.0,1,0,7.2500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,71.2833,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1,"Heikkinen, Miss. Laina",26.0,0,0,7.9250,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,53.1000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0,"Allen, Mr. William Henry",35.0,0,0,8.0500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [37]:
encod.get_feature_names()

In [90]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
x = df[['Age','SibSp','Parch','Fare',*encod.get_feature_names()]].values
y = df[['Survived']].values  
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.10,random_state=42)

clf = LogisticRegression().fit(x_train, y_train)
clf.predict(x_test)

print(clf.score(x_test, y_test))
#clf.coef_

#teste='x0_1','x0_2','x0_3','x1_female','x1_male','x2_C','x2_Q','x2_S'

/home/marcus/snap/jupyter/common/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/marcus/snap/jupyter/common/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: X has 1 features per sample; expecting 12

In [88]:
clf.predict_proba(x_test)

array([[0.89677543, 0.10322457],
       [0.05719782, 0.94280218],
       [0.24904313, 0.75095687],
       [0.72398615, 0.27601385],
       [0.05251993, 0.94748007],
       [0.89336696, 0.10663304],
       [0.68405649, 0.31594351],
       [0.86891619, 0.13108381],
       [0.08634479, 0.91365521],
       [0.04355622, 0.95644378],
       [0.59784334, 0.40215666],
       [0.43788959, 0.56211041],
       [0.95462118, 0.04537882],
       [0.87295689, 0.12704311],
       [0.89992792, 0.10007208],
       [0.8935271 , 0.1064729 ],
       [0.64460014, 0.35539986],
       [0.85307638, 0.14692362],
       [0.70269127, 0.29730873],
       [0.90365726, 0.09634274],
       [0.69814473, 0.30185527],
       [0.08088743, 0.91911257],
       [0.94014357, 0.05985643],
       [0.41939691, 0.58060309],
       [0.90362558, 0.09637442],
       [0.51460552, 0.48539448],
       [0.67120817, 0.32879183],
       [0.13122058, 0.86877942],
       [0.53028606, 0.46971394],
       [0.69354477, 0.30645523],
       [0.

In [55]:
clf.classes_

array([0, 1])